# Pruning puf2011 for regression

In [183]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import json as js
import re

In [184]:
#Loads variable meta-data from https://github.com/andersonfrailey/taxdata/blob/f17d6053d09536e7c2e198a6640c2bd0843deeb0/tests/records_metadata.json
true = 'true'
meta0 = open('records_metadata.json')
meta = js.load(meta0)
pufcps = []

#Creates list containing those variables available in both cps and puf by including those variables who have two elements in their 'availability' section in the JSON metadata
for i in meta:
    if len(meta[i]['availability'].split(',')) == 2:
        pufcps.append(i)

#Many variables shared between puf/cps are not in puf2011 and cannot be used in puf2011 -> cps imputation, these unshared variables are listed so they may be removed
notpuf2011 = ['agi_bin','age_head','age_spouse','e00200p','e00200s', 'e00900p','e00900s','e02100p','e02100s','elderly_dependent','filer','fips', 'nu05','nu13','nu18','n1820','n21']

#Creates list of variables in both puf2011 and cps
puf2011cps = [x for x in pufcps if x not in notpuf2011]

#Creates dictionary containing meta-data on puf2011cps variables, pulled from JSON meta-data
meta_puf2011cps = {}
for i in meta:
    if i in puf2011cps:
        meta_puf2011cps[i] = meta[i]['desc'] 

#Capitalizes variables to align with puf2011's formatting
puf2011cps = [x.upper() for x in puf2011cps]

#Adds to puf2011 those variables which we are trying to impute to cps. By definition they are not in cps, but, we intend to keep them in puf2011 so we can model them
puf2011cps.extend(['E02000','E26270','P22250','P23250'])

meta_puf2011cps

{'DSI': '1 if claimed as dependent on another return; otherwise 0',
 'EIC': 'number of EIC qualifying children (range: 0 to 3)',
 'FLPDYR': 'Calendar year for which taxes are calculated',
 'MARS': 'Filing (marital) status: line number of the checked box [1=single, 2=joint, 3=separate, 4=household-head, 5=widow(er)]',
 'RECID': 'Unique numeric identifier for filing unit; appears as RECID variable in tc CLI minimal output',
 'XTOT': 'Total number of exemptions for filing unit',
 'e00200': 'Wages, salaries, and tips for filing unit',
 'e00300': 'Taxable interest income',
 'e00400': 'Tax-exempt interest income',
 'e00600': 'Ordinary dividends included in AGI',
 'e00650': 'Qualified dividends included in ordinary dividends',
 'e00800': 'Alimony received',
 'e00900': 'Sch C business net profit/loss for filing unit',
 'e01100': 'Capital gain distributions not reported on Sch D',
 'e01400': 'Taxable IRA distributions',
 'e01500': 'Total pensions and annuities',
 'e01700': 'Taxable pensions and

In [185]:
puf2011 = pd.read_csv('puf2011.csv')

#Drops colums that are not in puf2011cps
for i in puf2011.columns:
    if i not in puf2011cps:
        puf2011.drop(i, axis=1, inplace=True)

#Adds constant
puf2011['constant'] = np.ones(len(puf2011))
    
#Creates list of variables which I assume are not useful for imputation, or are the dependent variables
irrelevant = ['S006','N24','F2441','E32800','E17500','EIC','FLPDYR','MARS','RECID','XTOT', 'E02000', 'E26270', 'P22250', 'P23250']

#Creates list of variables that may be useful as independent variables in imputation
independents = [x for x in puf2011cps if x not in irrelevant]

#Creates sub-dataframes where variables of interest are non-zero
pufE02 = puf2011[(puf2011['E02000']!=0)]
pufE26 = puf2011[(puf2011['E26270']!=0)]
pufP22 = puf2011[(puf2011['P22250']!=0)]
pufP23 = puf2011[(puf2011['P23250']!=0)]

# Models & parameter extraction

In [186]:
E02000_model = sm.OLS(pufE02['E02000'], 
                      pufE02[['E00200','E00300','E18400','E18500','E00600',
                              'E00650','E03300','E20400','E19800','E20100','constant']].dropna()).fit()
E02000_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 E02000   R-squared:                       0.198
Model:                            OLS   Adj. R-squared:                  0.198
Method:                 Least Squares   F-statistic:                     1387.
Date:                Fri, 25 May 2018   Prob (F-statistic):               0.00
Time:                        17:10:25   Log-Likelihood:            -8.8547e+05
No. Observations:               56321   AIC:                         1.771e+06
Df Residuals:                   56310   BIC:                         1.771e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
E00200        -0.1815      0.006    -31.793      0.000      -0.193      -0.170
E00300        -0.6333      0.020    -30.959      0.000      -0.673      -0.593
E18400         2.1596      0.025     87.594      0.000       2.111       2.208
E18500         7.1373      0.305     23.381      0.000       6.539       7.736
E00600        -0.1909      0.035     -5.514      0.000      -0.259      -0.123
E00650         0.0377      0.038      0.995      0.320      -0.037       0.112
E03300         4.8130      0.318     15.155      0.000       4.191       5.436
E20400        -1.5154      0.072    -21.148      0.000      -1.656      -1.375
E19800         1.1460      0.025     45.673      0.000       1.097       1.195
E20100         0.3908      0.023     16.969      0.000       0.346       0.436
constant    1.419e+05   8086.334     17.551      0.000    1.26e+05    1.58e+05
==============================================================================
Omnibus:                    48390.779   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         74090266.969
Skew:                           2.840   Prob(JB):                         0.00
Kurtosis:                     180.594   Cond. No.                     1.64e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.64e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [187]:
E26270_model = sm.OLS(pufE26['E26270'], 
                      pufE26[['E00200','E00300','E18400','E18500','E00600',
                               'E00650','E03300','E20400','E19800','E20100','constant']].dropna()).fit()
E26270_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 E26270   R-squared:                       0.185
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                     1015.
Date:                Fri, 25 May 2018   Prob (F-statistic):               0.00
Time:                        17:10:26   Log-Likelihood:            -7.0814e+05
No. Observations:               44811   AIC:                         1.416e+06
Df Residuals:                   44800   BIC:                         1.416e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
E00200        -0.1723      0.006    -27.143      0.000      -0.185      -0.160
E00300        -0.6443      0.022    -28.937      0.000      -0.688      -0.601
E18400         2.0707      0.027     77.032      0.000       2.018       2.123
E18500         6.0937      0.339     17.970      0.000       5.429       6.758
E00600        -0.1829      0.038     -4.784      0.000      -0.258      -0.108
E00650         0.0206      0.042      0.494      0.622      -0.061       0.103
E03300         5.2299      0.353     14.826      0.000       4.539       5.921
E20400        -1.3868      0.078    -17.672      0.000      -1.541      -1.233
E19800         1.0267      0.027     37.583      0.000       0.973       1.080
E20100         0.3524      0.025     14.082      0.000       0.303       0.401
constant    1.609e+05   1.01e+04     15.929      0.000    1.41e+05    1.81e+05
==============================================================================
Omnibus:                    36147.061   Durbin-Watson:                   1.950
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         45293363.017
Skew:                           2.552   Prob(JB):                         0.00
Kurtosis:                     158.667   Cond. No.                     1.85e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.85e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [188]:
P22250_model = sm.OLS(pufP22['P22250'], 
                      pufP22[['E00200','E00300','E18400','E18500','E00600',
                              'E03300','E20400', 'E00650','E19800','E20100','constant']].dropna()).fit()
P22250_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 P22250   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     21.23
Date:                Fri, 25 May 2018   Prob (F-statistic):           5.52e-40
Time:                        17:10:27   Log-Likelihood:            -6.5400e+05
No. Observations:               43055   AIC:                         1.308e+06
Df Residuals:                   43044   BIC:                         1.308e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
E00200         0.0123      0.003      3.949      0.000       0.006       0.018
E00300         0.1094      0.012      9.050      0.000       0.086       0.133
E18400        -0.0006      0.015     -0.038      0.970      -0.029       0.028
E18500        -0.4294      0.188     -2.281      0.023      -0.798      -0.060
E00600        -0.0294      0.020     -1.481      0.138      -0.068       0.010
E03300         0.1870      0.195      0.960      0.337      -0.195       0.569
E20400         0.1555      0.038      4.089      0.000       0.081       0.230
E00650         0.0467      0.022      2.145      0.032       0.004       0.089
E19800         0.0436      0.015      2.953      0.003       0.015       0.073
E20100        -0.0664      0.013     -4.954      0.000      -0.093      -0.040
constant   -2.808e+04   5637.124     -4.982      0.000   -3.91e+04    -1.7e+04
==============================================================================
Omnibus:                   141766.622   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     100506477916.838
Skew:                         -56.244   Prob(JB):                         0.00
Kurtosis:                    7487.138   Cond. No.                     2.13e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [189]:
P23250_model = sm.OLS(pufP23['P23250'], 
                      pufP23[['E00200','E00300','E18400','E18500','E00600',
                              'E00650','E03300','E20400','E19800','E20100','constant']].dropna()).fit()
P23250_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 P23250   R-squared:                       0.177
Model:                            OLS   Adj. R-squared:                  0.176
Method:                 Least Squares   F-statistic:                     1186.
Date:                Fri, 25 May 2018   Prob (F-statistic):               0.00
Time:                        17:10:33   Log-Likelihood:            -8.8832e+05
No. Observations:               55328   AIC:                         1.777e+06
Df Residuals:                   55317   BIC:                         1.777e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
E00200        -0.1204      0.007    -16.593      0.000      -0.135      -0.106
E00300         0.4575      0.028     16.162      0.000       0.402       0.513
E18400         1.5091      0.034     44.419      0.000       1.442       1.576
E18500         4.8852      0.420     11.621      0.000       4.061       5.709
E00600         0.5817      0.047     12.403      0.000       0.490       0.674
E00650        -0.3466      0.051     -6.749      0.000      -0.447      -0.246
E03300        -2.8935      0.435     -6.659      0.000      -3.745      -2.042
E20400         2.9221      0.090     32.633      0.000       2.747       3.098
E19800         0.5144      0.034     15.163      0.000       0.448       0.581
E20100         0.3320      0.031     10.578      0.000       0.270       0.394
constant     1.27e+05   1.16e+04     10.958      0.000    1.04e+05     1.5e+05
==============================================================================
Omnibus:                   104752.450   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        317107908.708
Skew:                          14.445   Prob(JB):                         0.00
Kurtosis:                     372.756   Cond. No.                     1.87e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.87e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [190]:
#Extract all parameters
betas = pd.DataFrame({'E02000_model':E02000_model.params,
                      'E26270_model':E26270_model.params,
                      'P22250_model':P22250_model.params,
                      'P23250_model':P23250_model.params}).fillna(0.)
betas.to_csv('Data_betas.csv')

#Revert variable names to lowercase to impute to cps data
betas.index = betas.index.str.lower()
betas.columns = betas.columns.str.lower()
betas

,e02000_model,e26270_model,p22250_model,p23250_model
e00200,-0.181507,-0.172296,0.012279,-0.120431
e00300,-0.633307,-0.644304,0.109411,0.457497
e00600,-0.190905,-0.182869,-0.029420,0.581737
e00650,0.037722,0.020643,0.046703,-0.346634
e03300,4.813039,5.229932,0.186951,-2.893479
e18400,2.159597,2.070738,-0.000558,1.509083
e18500,7.137264,6.093653,-0.429419,4.885248
e19800,1.145951,1.026750,0.043633,0.514401
e20100,0.390836,0.352398,-0.066435,0.332017
e20400,-1.515352,-1.386823,0.155526,2.922101


# Imputation

In [193]:
#Creates list of dependent variables, fetches cps data, adds constant column for imputation
dependents = ['e02000','e26270','p22250','p23250']
cps = pd.read_csv('cps.csv')
cps['constant'] = 1

#Creates new column for each dependent variable. Value in new column is the dot product of the row-vector containing 
# the independent variables' values for the given row and a column vector comprising the regression coefficients corresponding
# to the given dependent variable. As the coefficients are in the same order in their column as their independent 
# variables' values in this row, the result is a linear combination of our independent variable values with their corresponding 
# coefficient from our regression.

for i in dependents:
    #Creates row vector comprising our coefficients for the given model
    colv = np.array(betas[i+'_model'])
    #Reshapes into column vector
    colv = colv.reshape(11,1)
    #Dot product of independent variables' values & coefficient column vector.
    cps[i] = np.dot(np.array(cps[['e00200', 'e00300', 'e00600', 
                                    'e00650', 'e03300', 'e18400', 
                                    'e18500', 'e19800', 'e20100', 
                                    'e20400', 'constant']]), colv)
cps

,age_head,age_spouse,e00200p,e00900p,e02100p,e00200s,e00900s,e02100s,a_lineno,e00600,...,e03150,agi_bin,other_ben,e00200,e02100,constant,e02000,e26270,p22250,p23250
0,48,45,36428,0,0,5464,0,0,1,0,...,0,8,0,41892,0,1,158993.286367,175523.206176,-28263.961246,137677.242398
1,23,0,14571,0,0,0,0,0,3,0,...,0,4,0,14571,0,1,141009.348562,160052.970279,-27905.961657,126423.912437
2,80,0,0,0,0,0,0,0,1,0,...,0,3,17386,0,0,1,143202.726341,162130.713689,-28084.757360,127863.313274
3,64,0,20035,0,0,0,0,0,1,0,...,0,5,214,20035,0,1,155758.216417,172839.400337,-28312.391883,135473.351911
4,71,67,0,0,0,0,0,0,1,0,...,0,4,0,0,0,1,154687.683869,172011.450807,-28732.934767,135734.659621
5,80,85,0,0,0,0,0,0,1,0,...,0,9,0,0,0,1,168311.811545,184151.072483,-29263.227445,145129.104233
6,46,60,30053,0,0,68302,0,0,1,0,...,0,11,0,98355,0,1,162229.179330,178033.511315,-27899.323211,140217.406044
7,63,61,0,17367,0,10928,0,0,1,0,...,0,8,0,10928,0,1,166537.143972,182510.209269,-28752.619334,142746.698479
8,58,0,0,64548,0,0,0,0,1,0,...,0,1,363,0,0,1,145779.918896,164224.645211,-28300.493026,129612.833340
9,27,27,45535,0,0,910,0,0,1,0,...,0,9,7316,46445,0,1,144849.988224,163425.820062,-27377.603788,128013.683952


In [158]:
#Manual run to confirm results of preceding dot product results are identical to one decimal place, which I assume is a rounding difference.

cps = pd.read_csv('cps.csv')
cps['constant'] = 1

cps['e02000'] = (cps['e00200']*(-0.181507) + cps['e00300']*(-0.633307) 
              + cps['e00600']*(-0.190905) + cps['e00650']*(0.037722) 
              + cps['e03300']*(4.813039) + cps['e18400']*(2.159597) 
              + cps['e18500']*(7.137264) + cps['e19800']*(1.145951) 
              + cps['e20100']*(0.390836) + cps['e20400']*(-1.515352) 
              + cps['constant']*(141926.404421))
 
cps['e26270'] = (cps['e00200']*(-0.172296) + cps['e00300']*(-0.644304) 
              + cps['e00600']*(-0.182869) + cps['e00650']*(0.020643) 
              + cps['e03300']*(5.229932) + cps['e18400']*(2.070738) 
              + cps['e18500']*(6.093653) + cps['e19800']*(1.026750) 
              + cps['e20100']*(0.352398) + cps['e20400']*(-1.386823) 
              + cps['constant']*(160906.907562))

cps['p22250'] = (cps['e00200']*(0.012279) + cps['e00300']*(0.109411) 
              + cps['e00600']*(-0.029420) + cps['e00650']*(0.046703) 
              + cps['e03300']*(0.186951) + cps['e18400']*(-0.000558) 
              + cps['e18500']*(-0.429419) + cps['e19800']*(0.043633) 
              + cps['e20100']*(-0.066435) + cps['e20400']*(0.155526) 
              + cps['constant']*(-28084.427377))

cps['p23250'] = (cps['e00200']*(-0.120431) + cps['e00300']*(0.457497) 
              + cps['e00600']*(0.581737) + cps['e00650']*(-0.346634) 
              + cps['e03300']*(-2.893479) + cps['e18400']*(1.509083) 
              + cps['e18500']*(4.885248) + cps['e19800']*(0.514401) 
              + cps['e20100']*(0.332017) + cps['e20400']*(2.922101) 
              + cps['constant']*(126971.445257))

cps

,age_head,age_spouse,e00200p,e00900p,e02100p,e00200s,e00900s,e02100s,a_lineno,e00600,...,e03150,agi_bin,other_ben,e00200,e02100,constant,e02000,e26270,p22250,p23250
0,48,45,36428,0,0,5464,0,0,1,0,...,0,8,0,41892,0,1,158993.274204,175523.214977,-28263.945042,137677.238231
1,23,0,14571,0,0,0,0,0,3,0,...,0,4,0,14571,0,1,141009.343524,160052.972946,-27905.956468,126423.911556
2,80,0,0,0,0,0,0,0,1,0,...,0,3,17386,0,0,1,143202.726248,162130.713720,-28084.757155,127863.313310
3,64,0,20035,0,0,0,0,0,1,0,...,0,5,214,20035,0,1,155758.211434,172839.405112,-28312.383471,135473.349393
4,71,67,0,0,0,0,0,0,1,0,...,0,4,0,0,0,1,154687.684231,172011.450375,-28732.935124,135734.659015
5,80,85,0,0,0,0,0,0,1,0,...,0,9,0,0,0,1,168311.811968,184151.071777,-29263.227578,145129.103222
6,46,60,30053,0,0,68302,0,0,1,0,...,0,11,0,98355,0,1,162229.148114,178033.530361,-27899.286936,140217.398079
7,63,61,0,17367,0,10928,0,0,1,0,...,0,8,0,10928,0,1,166537.142317,182510.212404,-28752.613664,142746.696189
8,58,0,0,64548,0,0,0,0,1,0,...,0,1,363,0,0,1,145779.919047,164224.645057,-28300.493210,129612.833127
9,27,27,45535,0,0,910,0,0,1,0,...,0,9,7316,46445,0,1,144849.973829,163425.830310,-27377.585106,128013.680356


In [44]:
#P22250 model which utilizes the other variables we are trying to impute/predict, I may use this model instead

P22250_modelB = sm.OLS(pufP22['P22250'], 
                       pufP22[['E03300','E18400','E18500','E00200','E02000',
                               'E26270','E18500','P23250','E20400','E00650','E19800','E20100']].dropna()).fit()
P22250_modelB.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 P22250   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     40.94
Date:                Fri, 25 May 2018   Prob (F-statistic):           3.78e-89
Time:                        12:08:23   Log-Likelihood:            -6.5389e+05
No. Observations:               43055   AIC:                         1.308e+06
Df Residuals:                   43044   BIC:                         1.308e+06
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
E03300         0.2461      0.192      1.283      0.199      -0.130       0.622
E18400         0.1329      0.015      8.621      0.000       0.103       0.163
E18500        -0.1878      0.086     -2.178      0.029      -0.357      -0.019
E00200        -0.0014      0.003     -0.447      0.655      -0.007       0.005
E02000         0.0143      0.012      1.206      0.228      -0.009       0.038
E26270        -0.0654      0.012     -5.352      0.000      -0.089      -0.041
E18500        -0.1878      0.086     -2.178      0.029      -0.357      -0.019
P23250        -0.0103      0.002     -5.377      0.000      -0.014      -0.007
E20400         0.1946      0.037      5.262      0.000       0.122       0.267
E00650         0.0138      0.008      1.688      0.091      -0.002       0.030
E19800         0.1101      0.015      7.364      0.000       0.081       0.139
E20100        -0.0398      0.013     -2.967      0.003      -0.066      -0.013
==============================================================================
Omnibus:                   142039.393   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     101482188578.474
Skew:                         -56.565   Prob(JB):                         0.00
Kurtosis:                    7523.377   Cond. No.                     4.83e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.4e-18. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""